In [1]:
#|default_exp datadownload

In [193]:
from pathlib import Path
import requests
import csv
from io import StringIO
from rich import inspect
import pandas as pd
import json
import duckdb

# Data download

In [2]:
def extract_metadata(metadata_url):
    """
    Extracts a list of filenames and urls from Open Cananda metadata url.

    Parameters
    ----------
    metadata_url : str
        Fuel consumption ratings metadata url from Open Canada website.

    Returns
    -------
    english_resources_df : pd.DataFrame
        DataFrame of file names and urls for energy consumption ratings.
    """
    try:      
        metadata_resp = requests.get(metadata_url)
    except requests.exceptions.RequestException as e:
        # If request fails, return an error message and stop.
        print(f'Error making url request: {e}')
    
    try:     
        metadata_json = metadata_resp.json()
    except json.JSONDecodeError:
        # If parsing json fails, return an error message and stop.
        print(f'Error: Response is not valid json')
        
    # Access list of downloadable resources
    resources_df = pd.DataFrame(metadata_json['result']['resources'])

    # Change language coding and extract English only resources
    resources_df['language'] = resources_df['language'].apply(lambda item : item[0])
    english_resources_df = resources_df[resources_df['language'] == 'en']
    
    return english_resources_df[['name', 'url']]

In [3]:
#|export
def extract_raw_data(url, file_name):
    """
    Extract raw data from a URL

    Parameters
    ----------
    url : str
        URL to extract data from
        
    file_name : str or Path object
        file name for raw data dump
        
    """
    
    try:
        # Request data from url
        response = requests.get(url)
        content_type = response.headers['content-type']
        response_text = response.text
        print(f'Response status: {response.status_code}\nContent Type: {content_type}')

        # Save request content to csv file
        with open(file_name, mode='w', newline='') as csvfile:
            csvfile.write(response_text)

        print(f'csv file: {file_name} saved')
        
    # Catch errors    
    except requests.exceptions.HTTPError as err_h:
        print(f'HTTP error occured:{err_h}')
    except requests.exceptions.ConnectionError as err_c:
        print(f'Error connecting:{err_c}')
    except requests.exceptions.Timeout as err_t:
        print(f'Timeout Error:{err_t}')
    except requests.exceptions.RequestException as err:
        print(f'There was an unknown error:{err}')

In [4]:
def merge_top_two_rows(input_file, output_file):
    # Open the input CSV file for reading
    with open(input_file, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        
        # Read the first two rows from the input file
        header_row = next(reader)
        second_row = next(reader)
        
        # Merge the two rows into one header
        merged_header = [f"{header_row[i]} {second_row[i]}" for i in range(len(header_row))]
        
        # Open the output CSV file for writing
        with open(output_file, mode='w', newline='') as output_csvfile:
            writer = csv.writer(output_csvfile)
            
            # Write the merged header to the output file
            writer.writerow(merged_header)
            
            # Copy the rest of the rows from the input file to the output file
            for row in reader:
                writer.writerow(row)

In [124]:
def rename_columns(df):
    """
    Removes unwanted DataFrame columns and rows, then cleans and renames column headers.

    Parameters
    ----------
    df: DataFrame
        DataFrame with columns to clean

    Returns
    -------
    df: DataFrame
        DataFrame with cleaned column headers

    """
    # Drop empty columns and rows from the DataFrame
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, thresh=5, inplace=True)

    # Remove whitespace, replace spaces with _ and change to lower case
    cleaned_cols = (df.columns.str.lower()
                    .str.strip()
                    .str.replace(' # = high output engine', '')
                    .str.replace('*', '')
                    .str.replace('  ', ' ')
                    .str.replace(' ', '_')
                    .str.replace('(', '')
                    .str.replace(')', '')
                    .str.replace('/', '_')
    )

    col_mapper = dict(list(zip(df.columns, cleaned_cols))) # build a dictionary to map old column names to new
    df.rename(columns=col_mapper, inplace=True)
    return df  

In [195]:
def clean_content(df):
    """
    Clean content of master_df columns

    Parameters
    ----------
    df : DataFrame
        Should be master_df containing all fuel rating data combined
    """
    
    # Set fuel type columns using fuel_dict
    df['fuel_type'] = df['fuel_type'].map(fuel_dict)
    df['fuel_type_1'] = df['fuel_type_1'].map(fuel_dict)
    df['fuel_type_2'] = df['fuel_type_2'].map(fuel_dict)

    # Set make, model and vehicle_class to lower case and remove ":" characters
    df['make'] = df['make'].str.lower().str.strip()
    df['model'] = df['model'].str.lower().str.strip()
    df['vehicle_class'] = df['vehicle_class'].str.lower().str.strip()
    df['vehicle_class'] = df['vehicle_class'].str.replace(":", "-")

    # Set make, model and vehicle_class to category columns
    df['make'] = df['make'].astype('category')
    df['model'] = df['model'].astype('category')
    df['vehicle_class'] = df['vehicle_class'].astype('category')

    # Split transmission column into transmission type and number of gears
    df = df.join(df['transmission'].str.split(r"(\d+)", expand=True)
                          .drop(columns=[2])
                          .rename(columns={0: 'transmission_type', 1: 'number_of_gears'})
    )

    df['transmission_type'] = df['transmission_type'].map(transmission_dict)
    df.drop(columns='transmission', inplace=True)

    return df

In [213]:
def init_duckdb(db_file_path):
    """
    Initialize a DuckDB data base and create tables for each DataFrame

    Parameters
    ----------
    db_file_path : str
        Path to the DuckDB database file
    """
    db_connection = duckdb.connect(db_file_path)
    create_duckdb_table(db_connection, "all_vehicles", "master_df")

In [203]:
def create_duckdb_table(db_connection, table_name, df):
    """
    Create a table in DuckDB

    Parameters
    ----------
    db_connection : duckdb.conect
        Connection to DuckDB
    table_name : str
        Name of the table to be created
    df : str
        Nmae of the DataFrame to be used to create the table.
    """
    db_connection.execute(f"DROP TABLE IF EXISTS {table_name}")
    db_connection.execute(f"CREATE TABLE {table_name} AS SELECT * FROM {df}")

In [201]:
??duckdb.connect

Docstring:
connect(database: str = ':memory:', read_only: bool = False, config: dict = None) -> duckdb.DuckDBPyConnection

Create a DuckDB database instance. Can take a database file name to read/write persistent data and a read_only flag if no changes are desired
Type:      builtin_function_or_method

In [175]:
global model_dict
global transmission_dict
global fuel_dict

model_dict = {
    "4wd/4X4": "Four-wheel drive",
    "awd": "All-wheel drive",
    "ffv": "Flexible-fuel vehicle",
    "swb": "Short wheelbase",
    "lwb": "Long wheelbase",
    "ewb": "Extended wheelbase",
    "cng": "Compressed natural gas",
    "ngv": "Natural gas vehicle",
    "#": "High output engine that \
            provides more power than the standard \
            engine of the same size",
}

transmission_dict = {
    "A": "automatic",
    "AM": "automated manual",
    "AS": "automatic with select Shift",
    "AV": "continuously variable",
    "M": "manual",
}

fuel_dict = {
    "X": "regular gasoline",
    "Z": "premium gasoline",
    "D": "diesel",
    "E": "ethanol (E85)",
    "N": "natural gas",
    "B": "electricity",
    "B/X": "electricity & regular gasoline",
    "B/Z": "electricity & premium gasoline",
    "B/Z*": "electricity & premium gasoline",
    "B/X*": "electricity & regular gasoline",
    "B": "electricity",
}

## Script

In [9]:
url = 'https://natural-resources.canada.ca/sites/nrcan/files/oee/files/csv/MY2023%20Fuel%20Consumption%20Ratings.csv'
metadata_url = 'https://open.canada.ca/data/api/action/package_show?id=98f1a129-f628-4ce4-b24d-6f16bf24dd64'

In [74]:
# Build list of available resources
resources_df = extract_metadata(metadata_url)

# Remove unwanted old resources
resources_df = resources_df[~resources_df['name'].str.contains('Original')]

# Build filenames for desired resources and add to resources_df
file_names = (resources_df['name']
     .str.replace(' ', '_')
     .str.replace('(', 'v_')
     .str.replace(')', '')
     .str.lower()
)
resources_df.loc[:,'file_name'] = file_names

# Build raw data file path
path = Path.cwd()
raw_path = path / 'data' / 'raw'
merged_header_path = path / 'data' / 'merged-headers'

# Download and save raw data for each resource
for idx, row in resources_df.iterrows():
    url = row[1]
    file_name = row[2]
    raw_file_name = raw_path / f'{file_name}.csv'
    merged_header_file_name = merged_header_path / f'{file_name}.csv'
    extract_raw_data(url, raw_file_name)
    merge_top_two_rows(raw_file_name, merged_header_file_name)

In [196]:
# Start a list of column headers and initiate a master_df
union_of_headers = set()
master_df = pd.DataFrame()

# Build the master DataFrame
for idx, row in resources_df.iterrows():
    # Open each csv file
    url = row[1]
    file_name = row[2]
    merged_header_file_name = merged_header_path / f'{file_name}.csv'

    # Rename the columns
    df = pd.read_csv(merged_header_file_name)
    df = rename_columns(df)

    # Add vehicle type based on file_name
    if 'hybrid' in file_name:
        df['vehicle_type'] = 'hybrid'
    elif 'electric' in file_name and 'hybrid' not in file_name:
        df['vehicle_type'] = 'electric'
    else:
        df['vehicle_type'] = 'fuel-only'
        
    # Add any missing column headers to master DataFrame columns
    union_of_headers = set.union(union_of_headers, set(df.columns))
    missing_cols = set(master_df.columns) - union_of_headers
    if len(missing_cols) > 0:
        for col in missing_cols:
            master_df[col] = pd.Series()
        
    # Concatenate current df with master_df
    master_df = pd.concat([master_df, df], ignore_index=True)
  
    
master_df.head()

/tmp/ipykernel_2989/321982203.py:13: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(merged_header_file_name)
/tmp/ipykernel_2989/321982203.py:13: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(merged_header_file_name)
/tmp/ipykernel_2989/321982203.py:13: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(merged_header_file_name)
/tmp/ipykernel_2989/321982203.py:13: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(merged_header_file_name)
/tmp/ipykernel_2989/321982203.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(merged_header_file_name)


,model_year,make,model,vehicle_class,motor_kw,transmission,fuel_type,consumption_city_kwh_100_km,hwy_kwh_100_km,comb_kwh_100_km,...,fuel_type_1,consumption_combined_le_100_km,range_1_km,fuel_type_2,consumption_city_l_100_km,hwy_l_100_km,comb_l_100_km,range_2_km,fuel_consumption_city_l_100_km,comb_mpg
0,2012,Mitsubishi,i-MiEV,Subcompact,49.0,A1,B,16.9,21.4,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,Nissan,LEAF,Mid-size,80.0,A1,B,19.3,23.0,21.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,Ford,Focus Electric,Compact,107.0,A1,B,19.0,21.1,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,Mitsubishi,i-MiEV,Subcompact,49.0,A1,B,16.9,21.4,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Nissan,LEAF,Mid-size,80.0,A1,B,19.3,23.0,21.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
# Clean the master_df
master_df = clean_content(master_df)

In [216]:
# Create separate fuel, electric, and hybrid DataFrames
master_df.loc[master_df['vehicle_type'] == 'electric'].dropna(axis=1, how='all')

,model_year,make,model,vehicle_class,motor_kw,fuel_type,consumption_city_kwh_100_km,hwy_kwh_100_km,comb_kwh_100_km,city_le_100_km,hwy_le_100_km,comb_le_100_km,range_km,co2_emissions_g_km,co2_rating,smog_rating,recharge_time_h,vehicle_type,transmission_type,number_of_gears
0,2012,mitsubishi,i-miev,subcompact,49.0,electricity,16.9,21.4,18.7,1.9,2.4,2.1,100.0,0.0,NaN,NaN,7.0,electric,automatic,1
1,2012,nissan,leaf,mid-size,80.0,electricity,19.3,23.0,21.1,2.2,2.6,2.4,117.0,0.0,NaN,NaN,7.0,electric,automatic,1
2,2013,ford,focus electric,compact,107.0,electricity,19.0,21.1,20.0,2.1,2.4,2.2,122.0,0.0,NaN,NaN,4.0,electric,automatic,1
3,2013,mitsubishi,i-miev,subcompact,49.0,electricity,16.9,21.4,18.7,1.9,2.4,2.1,100.0,0.0,NaN,NaN,7.0,electric,automatic,1
4,2013,nissan,leaf,mid-size,80.0,electricity,19.3,23.0,21.1,2.2,2.6,2.4,117.0,0.0,NaN,NaN,7.0,electric,automatic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,2023,volkswagen,id.4,suv- small,150.0,electricity,18.2,21.2,19.6,2.0,2.4,2.2,336.0,0.0,10.0,10.0,6.0,electric,automatic,1
451,2023,volkswagen,id.4 pro,suv- small,150.0,electricity,18.2,21.3,19.6,2.0,2.4,2.2,443.0,0.0,10.0,10.0,7.5,electric,automatic,1
452,2023,volkswagen,id.4 awd pro,suv- small,220.0,electricity,20.1,22.6,21.2,2.3,2.5,2.4,410.0,0.0,10.0,10.0,7.5,electric,automatic,1
453,2023,volvo,c40 recharge twin,suv- small,300.0,electricity,22.2,26.1,23.9,2.5,2.9,2.7,364.0,0.0,10.0,10.0,8.0,electric,automatic,1


In [217]:
master_df.loc[master_df['vehicle_type'] == 'hybrid'].dropna(axis=1, how='all')

,model_year,make,model,vehicle_class,motor_kw,co2_emissions_g_km,co2_rating,smog_rating,recharge_time_h,vehicle_type,...,fuel_type_1,consumption_combined_le_100_km,range_1_km,fuel_type_2,consumption_city_l_100_km,hwy_l_100_km,comb_l_100_km,range_2_km,transmission_type,number_of_gears
455,2012,chevrolet,volt,compact,111.0,54.0,NaN,NaN,4.0,hybrid,...,electricity,2.5 (22.3 kWh/100 km),56.0,premium gasoline,6.7,5.9,6.4,550.0,continuously variable,None
456,2013,chevrolet,volt,compact,111.0,45.0,NaN,NaN,4.0,hybrid,...,electricity,2.4 (21.4 kWh/100 km),61.0,premium gasoline,6.7,5.9,6.4,550.0,continuously variable,None
457,2013,ford,c-max energi,mid-size,35.0,80.0,NaN,NaN,2.5,hybrid,...,electricity & regular gasoline,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,regular gasoline,5.8,6.5,6.1,856.0,continuously variable,None
458,2013,ford,fusion energi,mid-size,35.0,80.0,NaN,NaN,2.5,hybrid,...,electricity & regular gasoline,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,regular gasoline,5.8,6.5,6.1,856.0,continuously variable,None
459,2013,toyota,prius plug-in hybrid,mid-size,60.0,101.0,NaN,NaN,1.5,hybrid,...,electricity & regular gasoline,2.5 ([18.0 kWh + 0.4 L]/100 km),18.0,regular gasoline,4.7,4.8,4.7,845.0,continuously variable,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,2023,volvo,s60 t8 awd recharge,compact,107.0,58.0,10.0,7.0,5.0,hybrid,...,electricity & premium gasoline,3.0 ([27.2 kWh + 0.0 L]/100 km),64.0,premium gasoline,8.0,7.2,7.6,792.0,automatic with select Shift,8
701,2023,volvo,s90 t8 awd recharge,mid-size,107.0,65.0,10.0,7.0,5.0,hybrid,...,electricity & premium gasoline,3.4 ([30.0 kWh + 0.0 L]/100 km),61.0,premium gasoline,8.5,7.6,8.1,748.0,automatic with select Shift,8
702,2023,volvo,v60 t8 awd recharge,station wagon- small,107.0,58.0,10.0,7.0,5.0,hybrid,...,electricity & premium gasoline,3.0 ([27.2 kWh + 0.0 L]/100 km),64.0,premium gasoline,8.0,7.2,7.6,792.0,automatic with select Shift,8
703,2023,volvo,xc60 t8 awd recharge,suv- small,107.0,72.0,10.0,7.0,5.0,hybrid,...,electricity & premium gasoline,3.5 ([31.2 kWh + 0.0 L]/100 km),58.0,premium gasoline,8.5,8.5,8.5,838.0,automatic with select Shift,8


In [198]:
master_df.columns

Index(['model_year', 'make', 'model', 'vehicle_class', 'motor_kw', 'fuel_type',
       'consumption_city_kwh_100_km', 'hwy_kwh_100_km', 'comb_kwh_100_km',
       'city_le_100_km', 'hwy_le_100_km', 'comb_le_100_km', 'range_km',
       'co2_emissions_g_km', 'co2_rating', 'smog_rating', 'recharge_time_h',
       'vehicle_type', 'engine_size_l', 'cylinders', 'fuel_type_1',
       'consumption_combined_le_100_km', 'range_1_km', 'fuel_type_2',
       'consumption_city_l_100_km', 'hwy_l_100_km', 'comb_l_100_km',
       'range_2_km', 'fuel_consumption_city_l_100_km', 'comb_mpg',
       'transmission_type', 'number_of_gears'],
      dtype='object')

In [209]:
# Create directory for DuckDB database 
db_path = path / 'data' / 'database'
Path(db_path).mkdir(parents=True, exist_ok=True)
# Create file path for DuckDB database
db_file_path = str(db_path / 'car_data.duckdb')

In [211]:
type(db_file_path)

str

In [214]:
init_duckdb(db_file_path)

In [ ]:
# Export notebook to module.
from nbdev.export import nb_export
path = Path.cwd()
lib_path = path
nb_path = path / 'datadownload.ipynb'
nb_export(nb_path, lib_path)